In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date

In [2]:
config = configparser.ConfigParser()

#Normally this file should be in ~/.aws/credentials
config.read_file(open('dl.cfg'))

os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

In [3]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

# Song Data

In [4]:
songSchema = R([
            Fld("num_songs",Int()),
            Fld("artist_id",Str()),
            Fld("artist_latitude",Dbl()),
            Fld("artist_longitude",Dbl()),
            Fld("artist_location",Str()),
            Fld("artist_name",Str()),  
            Fld("title",Str()),  
            Fld("duration",Dbl()),  
            Fld("year",Int()),  
            ])

In [5]:
df = spark.read.json("s3a://udacity-dend/song_data/A/A/*/*.json", schema=songSchema)

In [6]:
df.printSchema()
df.show(5)
df.count()

root
 |-- num_songs: integer (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)

+---------+------------------+---------------+----------------+--------------------+--------------------+--------------------+----------+----+
|num_songs|         artist_id|artist_latitude|artist_longitude|     artist_location|         artist_name|               title|  duration|year|
+---------+------------------+---------------+----------------+--------------------+--------------------+--------------------+----------+----+
|        1|ARSUVLW12454A4C8B8|       35.83073|       -85.97874|           Tennessee|Royal Philharmoni...|Faust: Ballet Mus...|  94.56281|   0|
|        1|ARXQC081187FB4AD42|       54.

604

In [7]:
song_field = ["title", "duration", "year", "artist_id"]
songs_table = df.select(song_field).dropDuplicates().withColumn("song_id", F.monotonically_increasing_id()).filter(~col("year").isin([0]) & col("year").isNotNull() & col("artist_id").isNotNull())

In [8]:
songs_table.printSchema()
songs_table.show(5)
songs_table.count()

root
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- song_id: long (nullable = false)

+--------------------+---------+----+------------------+-----------+
|               title| duration|year|         artist_id|    song_id|
+--------------------+---------+----+------------------+-----------+
|Always And Never ...|  229.642|2005|AR10USD1187B99F3F1| 8589934592|
|    Commercial Reign|283.76771|1990|AR9AM2N1187B9AD2F1|17179869184|
|Terapia De Amor I...|340.29669|1988|ARBJSO81187B9BA09B|17179869187|
|Walking With The ...|152.16281|1966|ARE5F2F1187B9AB7E9|25769803776|
|               Pills| 256.1824|2000|ARW63XP1187FB5AB99|34359738369|
+--------------------+---------+----+------------------+-----------+
only showing top 5 rows



406

In [9]:
artist_field = ["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]
artists_table = df.select(artist_field).dropDuplicates().dropna(subset=["artist_id","artist_name"])

In [10]:
artists_table.printSchema()
artists_table.show(5)
artists_table.count()

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)

+------------------+-----------------+--------------------+---------------+----------------+
|         artist_id|      artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+-----------------+--------------------+---------------+----------------+
|AR1S3NH1187B98C2BC|        Anthony B|Clarks Town, Jamaica|           null|            null|
|ARPIKA31187FB4C233|       The Action|            New York|       40.71455|       -74.00712|
|ARYL56G11C8A41634E|    Mick Flannery|                    |           null|            null|
|AR1XL241187FB3F4AB|Nortec Collective|                    |           null|            null|
|ARMI4NV1187B99D55D|          Man Man|    Philadelphia, PA|       39.95227|       -75.16237|
+------------------+----

591

In [11]:
songs_table.write.partitionBy("year", "artist_id").parquet("s3a://sparkify-dend-test/songs/", mode="overwrite")

In [12]:
artists_table.write.parquet("s3a://sparkify-dend-test/artists/", mode="overwrite")

# Log Data

In [6]:
df = spark.read.json("s3a://udacity-dend/log_data/2018/11/*.json")

In [4]:
df = spark.read.json("./data/log_data/2018-11-01-events.json")

In [5]:
df.printSchema()
df.show(5)
df.count()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

+-------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+------------+------+-------------+--------------------+------+
| artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|meth

15

In [6]:
df = df.filter(df.page == 'NextSong')

In [7]:
df.printSchema()
df.show(5)
df.count()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

+-----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|           artist|     auth|firstName|gender|itemInSession|lastName|   length|lev

11

In [8]:
user_field = [" userId as user_id", "firstName as first_name", "lastName as last_name", "gender", "level"]
users_table = df.selectExpr(user_field).dropDuplicates().dropna(how = "any")

In [9]:
users_table.printSchema()
users_table.show(5)
users_table.count()

root
 |-- user_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     26|      Ryan|    Smith|     M| free|
|    101|    Jayden|      Fox|     M| free|
|     10|    Sylvie|     Cruz|     F| free|
|      8|    Kaylee|  Summers|     F| free|
+-------+----------+---------+------+-----+



4

In [43]:
users_table.write.parquet("s3a://sparkify-dend-test/users/", mode="overwrite")

In [10]:
df = df.withColumn(
    "start_time",
    F.to_timestamp(F.from_unixtime((col("ts") / 1000) , 'yyyy-MM-dd HH:mm:ss.SSS')).cast("Timestamp")
)

In [11]:
df.printSchema()
df.show(5)
df.count()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: timestamp (nullable = true)

+-----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+
|           artis

11

In [12]:
df = df.withColumn("test_time", F.to_timestamp(col("ts") / 1000))

In [13]:
df.printSchema()
df.show(5)
df.count()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- test_time: timestamp (nullable = true)

+-----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+-

11

In [14]:
time_table = df.select("start_time").dropDuplicates() \
.withColumn("hour", hour("start_time")).withColumn("day", dayofmonth("start_time")) \
.withColumn("week", weekofyear("start_time")).withColumn("month", month("start_time")) \
.withColumn("year", year("start_time")).withColumn("weekday", date_format("start_time", "E")).dropna(how="any")

In [15]:
time_table.printSchema()
time_table.show(5)
time_table.count()

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: string (nullable = true)

+-------------------+----+---+----+-----+----+-------+
|         start_time|hour|day|week|month|year|weekday|
+-------------------+----+---+----+-----+----+-------+
|2018-11-01 21:17:33|  21|  1|  44|   11|2018|    Thu|
|2018-11-01 21:05:52|  21|  1|  44|   11|2018|    Thu|
|2018-11-01 21:28:54|  21|  1|  44|   11|2018|    Thu|
|2018-11-01 21:52:05|  21|  1|  44|   11|2018|    Thu|
|2018-11-01 21:55:25|  21|  1|  44|   11|2018|    Thu|
+-------------------+----+---+----+-----+----+-------+
only showing top 5 rows



11

In [16]:
time_table.write.partitionBy("year", "month").parquet("s3a://sparkify-dend-test/time/", mode="overwrite")

In [ ]:
song_df = spark.read.parquet("s3a://sparkify-dend-test/songs/*/*/*")
artist_df = spark.read.parquet("s3a://sparkify-dend-test/artists/*")

In [19]:
song_df = spark.read.parquet("s3a://sparkify-dend-test/songs/*/*/*")

In [24]:
artist_df = spark.read.parquet("s3a://sparkify-dend-test/artists/*")

In [20]:
song_df.printSchema()

root
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- song_id: long (nullable = true)



In [25]:
Join_song = df.join(song_df, ((song_df.title == df.song) & (song_df.duration == df.length)))
artists_songs_logs = Join_song.join(artist_df, (Join_song.artist == artist_df.artist_name))
songplays = artists_songs_logs.join(time_table, (artists_songs_logs.start_time == time_table.start_time), 'left').drop(artists_songs_logs.start_time)

In [21]:
Join_song = df.join(song_df, ((song_df.title == df.song)))

In [22]:
Join_song.show(5)

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+----------+---------+-----+--------+-------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status| ts|userAgent|userId|start_time|test_time|title|duration|song_id|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+----------+---------+-----+--------+-------+
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+----------+---------+-----+--------+-------+



In [53]:
song_df.show(5)

+--------------------+---------+-------------+
|               title| duration|      song_id|
+--------------------+---------+-------------+
|Take It To Da Hou...|227.10812|1279900254208|
|Welcome To The Do...| 46.94159| 893353197568|
|The Czar: Usurper...|654.28853|1211180777472|
|N****_ What's Up ...|196.85832|1563368095745|
|I Think My Wife I...|186.48771| 326417514497|
+--------------------+---------+-------------+
only showing top 5 rows



In [1]:
artist_df.show(5)

NameError: name 'artist_df' is not defined

In [54]:
songplays.printSchema()
songplays.show(5)
songplays.count()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- test_time: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- song_id: long (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_lat

Py4JJavaError: An error occurred while calling o924.showString.
: org.apache.spark.SparkException: Could not execute broadcast in 300 secs. You can increase the timeout for broadcasts via spark.sql.broadcastTimeout or disable broadcast join by setting spark.sql.autoBroadcastJoinThreshold to -1
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:150)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:387)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql.exec

In [ ]:
songplays.count()

In [32]:
songplays_field = ["start_time", "userId as user_id", "level", "song_id", "artist_id", "sessionid as session_id", "artist_location as location", "userAgent as user_agent", "year as yeardata", "month as monthdata"]

In [33]:
songplays_table = songplays.selectExpr(songplays_field).dropDuplicates().dropna(subset=["user_id", "artist_id", \
"song_id"]).withColumn("songplay_id", F.monotonically_increasing_id())

In [ ]:
songplays_table.printSchema()
songplays_table.show(5)
songplays_table.count()

In [34]:
songplays_table.printSchema()

root
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: long (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- yeardata: integer (nullable = true)
 |-- monthdata: integer (nullable = true)
 |-- songplay_id: long (nullable = false)



In [35]:
songplays_table.write.partitionBy("yeardata", "monthdata").parquet("s3a://sparkify-dend-test/" + "songplays/", mode="overwrite")

Py4JJavaError: An error occurred while calling o287.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope

In [ ]:
songplays_table.createOrReplaceTempView("songplays")

spark.sql("""
    SELECT month, count(song_id) as song_num
    FROM songplays
    GROUP by month
    order by song_num desc
""").show()